In [1]:
import os
import cv2
import shutil
import random
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plot
from PIL import Image

## 组织数据

In [139]:
def gather_images(folder_path):
    image_dict = {}
    for dirpath, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                file_path = os.path.join(dirpath, filename)
                folder_name = os.path.basename(os.path.dirname(file_path))
                if folder_name not in image_dict:
                    image_dict[folder_name] = []
                image_dict[folder_name].append(file_path)
    return image_dict


def copy_images(image_dict, new_folder_path):
    for folder_name, file_paths in image_dict.items():
        for i, file_path in enumerate(file_paths):
            new_file_name = f"{folder_name}_{i+1}{os.path.splitext(file_path)[1]}"
            if not os.path.exists(os.path.join(new_folder_path, folder_name)):
                os.makedirs(os.path.join(new_folder_path, folder_name))
            new_file_path = os.path.join(new_folder_path, folder_name, new_file_name)
            shutil.copy(file_path, new_file_path)

In [140]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/chinese/新"
image_dict = gather_images(image_dir)

new_folder_path = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/chinese/新_1"
copy_images(image_dict, new_folder_path)

## 统计数据

In [1]:
def count_images_side(folder_path):
    images_side_dict = {}
    for dirpath, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                file_path = os.path.join(dirpath, filename)
                image = Image.open(file_path)
                width, height = image.size
                images_side_dict[file_path] = [width, height]

    # 根据模糊程度对图像进行排序
    sorted_images_side = sorted(
        images_side_dict.items(), key=lambda x: min(x[1]), reverse=True
    )
    return sorted_images_side


def plot_side_histogram(sorted_images_side):
    _, side_values = zip(*sorted_images_side)
    plot.rcParams["font.family"] = ["Times New Roman"]
    plot.figure(figsize=(10, 6))
    plot.hist(list(map(min, side_values)), bins=30, color="blue", alpha=0.7)
    plot.title("Image Side Distribution")
    plot.xlabel("Side")
    plot.ylabel("Frequency")
    plot.grid(True)
    plot.xticks(rotation=45)
    plot.minorticks_on()
    plot.tick_params(axis="both", which="both", direction="in")
    plot.show()

In [ ]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/chinese"
sorted_images_side = count_images_side(image_dir)
plot_side_histogram(sorted_images_side)

In [3]:
def count_images_total(folder_path):
    chinese_images = []
    nonchinese_images = []
    for dirpath, _, filenames in os.walk(folder_path):
        kind_name = os.path.basename(os.path.dirname(dirpath))
        if "nonchinese" not in kind_name:
            for filename in filenames:
                if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                    file_path = os.path.join(dirpath, filename)
                    chinese_images.append(file_path)
        elif "nonchinese" in kind_name:
            for filename in filenames:
                if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                    file_path = os.path.join(dirpath, filename)
                    nonchinese_images.append(file_path)
    print(f"chinese images total: {len(chinese_images)}")
    print(f"nonchinese images total: {len(nonchinese_images)}")

In [6]:
count_images_total(
    "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character"
)

chinese images total: 535942
nonchinese images total: 1049521


## 模糊模式

In [360]:
# 模糊程度排序
def estimate_blur_laplacian(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    laplacian_var = cv2.Laplacian(image, cv2.CV_64F).var()
    return laplacian_var


def sort_images_by_blur(folder_path):
    blur_dict = {}
    for dirpath, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                file_path = os.path.join(dirpath, filename)
                blur_measure = estimate_blur_laplacian(file_path)
                blur_dict[file_path] = blur_measure

    # 根据模糊程度对图像进行排序
    sorted_images = sorted(blur_dict.items(), key=lambda x: x[1], reverse=True)
    return sorted_images


def plot_blur_histogram(sorted_images):
    _, blur_values = zip(*sorted_images)
    plot.rcParams["font.family"] = ["Times New Roman"]
    plot.figure(figsize=(10, 6))
    plot.hist(blur_values, bins=30, color="blue", alpha=0.7)
    plot.title("Image Blur Distribution")
    plot.xlabel("Blur")
    plot.ylabel("Frequency")
    plot.grid(True)
    plot.xticks(rotation=45)
    plot.minorticks_on()
    plot.tick_params(axis="both", which="both", direction="in")
    plot.show()


def plot_blur_histogram_by_folder(sorted_images):
    folder_blur = {}
    for file_path, blur in sorted_images:
        # 提取直接文件夹的名称
        folder_name = os.path.basename(os.path.dirname(file_path))
        if folder_name not in folder_blur:
            folder_blur[folder_name] = []
        folder_blur[folder_name].append(blur)

    # 为每个文件夹绘制亮度直方图
    for folder_name, blur_values in folder_blur.items():
        plot.rcParams["font.family"] = ["STFangsong"]
        plot.rcParams["axes.unicode_minus"] = False
        plot.figure(figsize=(10, 6))
        plot.hist(blur_values, bins=30, color="blue", alpha=0.7)
        plot.title(f"Image Blur Distribution in {folder_name}")
        plot.xlabel("Blur")
        plot.ylabel("Frequency")
        plot.grid(True)
        plot.xticks(rotation=45)
        plot.minorticks_on()
        plot.tick_params(axis="both", which="both", direction="in")
        plot.show()


def combine_images(sorted_images):
    # 读取所有图像并存储它们及其尺寸
    images = []
    total_width = 0
    max_height = 0

    for image_path, _ in sorted_images:
        img = np.array(Image.open(image_path))
        images.append(img)
        h, w, _ = img.shape
        total_width += w
        max_height = max(max_height, h)

    # 创建一个足够大的画布
    combined_image = np.zeros((max_height, total_width, 3), dtype=np.uint8)

    # 将每张图像放置到画布上
    current_x = 0
    for img in images:
        h, w, _ = img.shape
        combined_image[:h, current_x : current_x + w, :] = img
        current_x += w

    return combined_image

In [361]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/chinese"
sorted_images = sort_images_by_blur(image_dir)

In [ ]:
plot_blur_histogram(sorted_images)
plot_blur_histogram_by_folder(sorted_images)

In [ ]:
combined_image = combine_images(sorted_images)
display(Image.fromarray(combined_image))

In [362]:
# 动态模糊（模糊核的越大，模糊程度越高，设置为图像尺寸的1/5到1/3）
def apply_motion_blur(image: np.ndarray, size=5):
    kernel_motion_blur = np.zeros((size, size))
    kernel_motion_blur[int((size - 1) / 2), :] = np.ones(size)
    kernel_motion_blur = kernel_motion_blur / size
    return cv2.filter2D(image, -1, kernel_motion_blur)

In [96]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/chinese"
sorted_images = sort_images_by_blur(image_dir)

In [405]:
image_data_pil = Image.open(sorted_images[2000][0])
display(
    image_data_pil.size, image_data_pil, estimate_blur_laplacian(sorted_images[2000][0])
)

image_processed = apply_motion_blur(np.array(image_data_pil), 9)
display(Image.fromarray(image_processed))

gray_image = cv2.cvtColor(image_processed, cv2.COLOR_RGB2GRAY)
laplacian_var = cv2.Laplacian(gray_image, cv2.CV_64F).var()
display(laplacian_var)

(15, 29)

22898.000285374554

836.3471211520676

In [408]:
# 对数据集中的所有图像应用模糊模式处理
def blur_dataset(image_dict, new_folder_path):
    for folder_name, file_paths in image_dict.items():
        if folder_name not in ["chinese", "nonchinese"]:
            nums = len(file_paths)
            for file_path in file_paths:
                image = np.array(Image.open(file_path))
                blur_measure = estimate_blur_laplacian(file_path)

                # 计算模糊核大小的范围
                min_side = min(image.shape[:2])
                max_kernel_size = int(min_side / 3)

                # 确保核大小为奇数
                init_size = 3
                size_gap = 2
                kernel_sizes = [init_size, init_size + size_gap]
                kernel_sizes.extend(
                    [
                        size
                        for size in range(3, max_kernel_size + 1, size_gap)
                        if size not in kernel_sizes
                        and size <= (init_size + 3 * size_gap)
                    ]
                )

                # 对每张图像应用不同大小的模糊核
                for size in kernel_sizes:
                    if nums > 5000 and size < (
                        init_size + size_gap
                    ):  # 数量够多的情况，跳过模糊度低的核
                        continue
                    if blur_measure < 10000 and size >= (
                        init_size + size_gap
                    ):  # 不够清晰的情况，跳过模糊度过高的核
                        continue
                    blurred_image = apply_motion_blur(image, size)
                    new_file_name = f"{os.path.splitext(os.path.basename(file_path))[0]}_blur{size}{os.path.splitext(file_path)[1]}"
                    new_file_path = os.path.join(
                        new_folder_path, folder_name, new_file_name
                    )

                    if not os.path.exists(os.path.join(new_folder_path, folder_name)):
                        os.makedirs(os.path.join(new_folder_path, folder_name))

                    Image.fromarray(blurred_image).save(new_file_path)

In [411]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/nonchinese"
image_dict = gather_images(image_dir)

blur_dataset(image_dict, f"{image_dir}_blur")

## 曝光模式

In [412]:
def calculate_yuv_brightness(image_path):
    image = cv2.imread(image_path)
    yuv_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    brightness = np.mean(yuv_image[:, :, 0])
    return brightness


def sort_images_by_brightness(folder_path):
    brightness_dict = {}
    for dirpath, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                file_path = os.path.join(dirpath, filename)
                brightness_measure = calculate_yuv_brightness(file_path)
                brightness_dict[file_path] = brightness_measure

    sorted_images = sorted(brightness_dict.items(), key=lambda x: x[1], reverse=True)
    return sorted_images


def plot_brightness_histogram(sorted_images):
    _, brightness_values = zip(*sorted_images)
    plot.rcParams["font.family"] = ["Times New Roman"]
    plot.rcParams["axes.unicode_minus"] = False
    plot.figure(figsize=(10, 6))
    plot.hist(brightness_values, bins=30, color="blue", alpha=0.7)
    plot.title("Image Brightness Distribution")
    plot.xlabel("Brightness")
    plot.ylabel("Frequency")
    plot.grid(True)
    plot.xticks(rotation=45)
    plot.minorticks_on()
    plot.tick_params(axis="both", which="both", direction="in")
    plot.show()


def plot_brightness_histogram_by_folder(sorted_images):
    folder_brightness = {}
    for file_path, brightness in sorted_images:
        # 提取直接文件夹的名称
        folder_name = os.path.basename(os.path.dirname(file_path))
        if folder_name not in folder_brightness:
            folder_brightness[folder_name] = []
        folder_brightness[folder_name].append(brightness)

    # 为每个文件夹绘制亮度直方图
    for folder_name, brightness_values in folder_brightness.items():
        plot.rcParams["font.family"] = ["STFangsong"]
        plot.rcParams["axes.unicode_minus"] = False
        plot.figure(figsize=(10, 6))
        plot.hist(brightness_values, bins=30, color="blue", alpha=0.7)
        plot.title(f"Image Brightness Distribution in {folder_name}")
        plot.xlabel("Brightness")
        plot.ylabel("Frequency")
        plot.grid(True)
        plot.xticks(rotation=45)
        plot.minorticks_on()
        plot.tick_params(axis="both", which="both", direction="in")
        plot.show()


def combine_images(sorted_images):
    images = []
    total_width = 0
    max_height = 0

    for image_path, _ in sorted_images:
        img = np.array(Image.open(image_path))
        images.append(img)
        h, w, _ = img.shape
        total_width += w
        max_height = max(max_height, h)

    combined_image = np.zeros((max_height, total_width, 3), dtype=np.uint8)

    current_x = 0
    for img in images:
        h, w, _ = img.shape
        combined_image[:h, current_x : current_x + w, :] = img
        current_x += w

    return combined_image

In [88]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/nonchinese"
sorted_images = sort_images_by_brightness(image_dir)

In [ ]:
plot_brightness_histogram(sorted_images)
plot_brightness_histogram_by_folder(sorted_images)

In [ ]:
combined_image = combine_images(sorted_images)
Image.fromarray(combined_image)

In [413]:
# 曝光（大于1.0会增加曝光，小于1.0会减少曝光，设置为0.1到3）
def adjust_exposure(image: np.ndarray, factor=1.0):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 2] = hsv[:, :, 2] * factor
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [101]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/chinese"
sorted_images = sort_images_by_brightness(image_dir)

In [115]:
image_data_pil = Image.open(sorted_images[200][0])
display(
    image_data_pil.size, image_data_pil, calculate_yuv_brightness(sorted_images[200][0])
)

image_processed = adjust_exposure(np.array(image_data_pil), 0.5)
display(Image.fromarray(image_processed))

yuv_image = cv2.cvtColor(image_processed, cv2.COLOR_BGR2YUV)
brightness = np.mean(yuv_image[:, :, 0])
display(brightness)

(19, 32)

235.1809210526316

120.95723684210526

In [415]:
# 对数据集中的所有图像应用曝光模式处理
def brightness_dataset(image_dict, new_folder_path):
    for folder_name, file_paths in image_dict.items():
        if folder_name not in ["chinese", "nonchinese"]:
            nums = len(file_paths)
            for file_path in file_paths:
                image = np.array(Image.open(file_path))
                brightness = calculate_yuv_brightness(file_path)

                # 设置亮度因子的范围
                factor_range = [0.1, 0.2, 0.5, 0.8, 1.5, 2.0, 3.0, 5.0]
                factor_range = [
                    factor + 0.1 * random.random() for factor in factor_range
                ]

                # 对每张图像应用不同大小的亮度因子
                selected_factor = random.sample(factor_range, 3)
                for factor in selected_factor:
                    if (brightness * factor > 225) or (
                        brightness * factor < 25
                    ):  # 跳过过亮或过暗的情况
                        continue
                    if brightness > 200 and factor > 0.2:  # 跳过图像过亮且要变得太暗的情况
                        continue
                    if brightness < 50 and factor < 3.0:  # 跳过图像过暗且要变得太亮的情况
                        continue
                    if (
                        nums > 10000 and (100 < brightness < 150) and (0.5 < factor < 2)
                    ):  # 跳过图像太多并且图像亮度合适但变化较小的情况
                        continue

                    brightness_image = adjust_exposure(image, factor)
                    new_file_name = f"{os.path.splitext(os.path.basename(file_path))[0]}_brightness{factor:.2f}{os.path.splitext(file_path)[1]}"
                    new_file_path = os.path.join(
                        new_folder_path, folder_name, new_file_name
                    )

                    if not os.path.exists(os.path.join(new_folder_path, folder_name)):
                        os.makedirs(os.path.join(new_folder_path, folder_name))

                    Image.fromarray(brightness_image).save(new_file_path)

                    if nums > 5000:  # 图像太多的情况做一次亮度调整就够了
                        break

In [417]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/nonchinese"
image_dict = gather_images(image_dir)

brightness_dataset(image_dict, f"{image_dir}_brightness")

## 其他模式（颜色抖动、噪声、遮挡、变换）

In [356]:
# 噪声（方差越大，噪声越大，设置为0.01到0.05）
def apply_add_noise(image: np.ndarray, threshold=15):
    row, col, ch = image.shape
    min_side = min([row, col])
    var_range = [0.01, 0.008, 0.005, 0.003]
    var = random.sample(var_range, 1)[0]
    if min_side < threshold:
        var = 0.003
    mean = 0
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = image.astype(np.float64) / 255 + gauss
    noisy = np.clip(noisy, 0, 1) * 255
    return noisy.astype(np.uint8)

In [227]:
# 颜色抖动
def apply_color_jitter(image: np.ndarray, threshold=15):
    min_side = min(image.shape[:2])
    image = Image.fromarray(image)
    if min_side < threshold:
        factor = 0.3
    else:
        factor = 0.5
    transform = transforms.ColorJitter(
        brightness=1, contrast=factor, saturation=factor, hue=factor
    )
    jittered_image = np.array(transform(image))
    return jittered_image

In [307]:
# 遮挡（遮挡值255为白色，0为黑色，128为灰色）
def apply_occlusion(image: np.ndarray, threshold=15):
    height, width, _ = image.shape
    min_side = min(image.shape[:2])
    min_area = height * width / 20
    max_area = height * width / 10
    if min_side < threshold:
        min_area = min_area / 1.5
        max_area = max_area / 1.5
    while True:
        if min_side < threshold:
            x, y = np.random.randint(width), np.random.randint(height)
        else:
            x, y = np.random.randint(0, width - 2), np.random.randint(0, height - 2)
        max_w, max_h = width - x, height - y
        if min_side < threshold:
            w, h = np.random.randint(1, max_w + 1), np.random.randint(1, max_h + 1)
        else:
            w, h = np.random.randint(3, max_w + 1), np.random.randint(3, max_h + 1)
        if min_area <= w * h <= max_area:
            break
    mask_range = [5, 50, 100, 150, 200, 250]
    mask = random.sample(mask_range, 1)[0]
    image_copy = image.copy()
    image_copy[y : y + h, x : x + w] = mask
    return image_copy

In [232]:
# 仿射变换（旋转30度、平移0.1、缩放2、倾斜0.5）
def apply_affine_transform(image: np.ndarray, threshold=15):
    # 获取图像尺寸
    height, width = image.shape[:2]
    min_side = min(image.shape[:2])
    # 计算旋转中心
    center = (width / 2, height / 2)

    # 参数随机化
    angle = np.random.randint(-15, 16)
    translate = (
        np.random.randint(-width // 20, width // 20 + 1),
        np.random.randint(-height // 10, height // 10 + 1),
    )
    scale = 0.85 + 0.3 * np.random.random()
    shear = np.random.uniform(-0.3, 0.3)
    if min_side < threshold:
        angle = np.random.randint(-10, 11)
        translate = (0, 0)
        scale = 1
        shear = np.random.uniform(-0.2, 0.2)

    # 计算旋转矩阵
    M_rotate = cv2.getRotationMatrix2D(center, angle, scale)
    M_rotate = np.vstack([M_rotate, [0, 0, 1]])
    # 计算倾斜变换矩阵
    M_shear = np.array([[1, shear, 0], [0, 1, 0], [0, 0, 1]])
    # 合并矩阵
    M = np.dot(M_rotate, M_shear)
    # 应用平移
    M[:2, 2] += translate
    # 应用仿射变换
    M_affine = M[:2, :]
    transformed = cv2.warpAffine(image, M_affine, (width, height))
    return transformed

In [8]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/chinese"
images_dict = gather_images(image_dir)

In [352]:
image_data_pil = Image.open(list(images_dict.values())[1][200])
display(image_data_pil.size, image_data_pil)

image_processed = apply_add_noise(np.array(image_data_pil), 15)
display(Image.fromarray(image_processed))

image_processed = apply_color_jitter(np.array(image_data_pil), 15)
display(Image.fromarray(image_processed))

image_processed = apply_occlusion(np.array(image_data_pil), 15)
display(Image.fromarray(image_processed))

image_processed = apply_affine_transform(np.array(image_data_pil), 15)
display(Image.fromarray(image_processed))

(8, 15)

In [357]:
# 对数据集中的所有图像应用其他模式处理
def other_dataset(image_dict, new_folder_path):
    for folder_name, file_paths in image_dict.items():
        if folder_name not in ["chinese", "nonchinese"]:
            nums = len(file_paths)
            k = 0
            for idx, file_path in enumerate(file_paths):
                image = np.array(Image.open(file_path))
                if not os.path.exists(os.path.join(new_folder_path, folder_name)):
                    os.makedirs(os.path.join(new_folder_path, folder_name))

                if nums > 4000:
                    if nums > 16000:
                        if idx % 8 != 0:  # 当 nums > 16000 时，只有当 idx 是 8 的倍数时才不跳过
                            continue
                    elif nums > 8000:
                        if (
                            idx % 4 != 0
                        ):  # 当 nums > 8000 但不超过 16000 时，只有当 idx 是 4 的倍数时才不跳过
                            continue
                    elif idx % 2 == 0:  # 当 nums > 4000 但不超过 8000 时，只有当 idx 是偶数时才跳过
                        continue

                noise_image = apply_add_noise(image, 15)
                k += 1
                new_file_name = f"{os.path.splitext(os.path.basename(file_path))[0]}_other{k}{os.path.splitext(file_path)[1]}"
                new_file_path = os.path.join(
                    new_folder_path, folder_name, new_file_name
                )
                Image.fromarray(noise_image).save(new_file_path)

                jitter_image = apply_color_jitter(image, 15)
                k += 1
                new_file_name = f"{os.path.splitext(os.path.basename(file_path))[0]}_other{k}{os.path.splitext(file_path)[1]}"
                new_file_path = os.path.join(
                    new_folder_path, folder_name, new_file_name
                )
                Image.fromarray(jitter_image).save(new_file_path)

                occlusion_image = apply_occlusion(image, 15)
                k += 1
                new_file_name = f"{os.path.splitext(os.path.basename(file_path))[0]}_other{k}{os.path.splitext(file_path)[1]}"
                new_file_path = os.path.join(
                    new_folder_path, folder_name, new_file_name
                )
                Image.fromarray(occlusion_image).save(new_file_path)

                transform_image = apply_affine_transform(image, 15)
                k += 1
                new_file_name = f"{os.path.splitext(os.path.basename(file_path))[0]}_other{k}{os.path.splitext(file_path)[1]}"
                new_file_path = os.path.join(
                    new_folder_path, folder_name, new_file_name
                )
                Image.fromarray(transform_image).save(new_file_path)

In [359]:
image_dir = "/Users/WangHao/Sites/学习/Python/TuLiProject/TrainNetPipeline/datas/character_advance/nonchinese"
image_dict = gather_images(image_dir)

other_dataset(image_dict, f"{image_dir}_other")